## Text summarization using spaCy

This is based on the following [article](https://medium.com/analytics-vidhya/text-summarization-using-spacy-ca4867c6b744).



It uses Extractive Summarization. These methods rely on extracting several parts, such as phrases and sentences, from a piece of text and stack them together to create a summary. Therefore, identifying the right sentences for summarization is of utmost importance in an extractive method.

In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

nlp = spacy.load('en_core_web_sm')

In [2]:
text = "Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task. Machine learning algorithms build a mathematical model of sample data, known as “training data”, in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a field of study within machine learning and focuses on exploratory data analysis through unsupervised learning. In its application across business problems, machine learning is also referred to as predictive analytics."

In [4]:
doc = nlp(text)

len(list(doc.sents))

7

### Filtering Tokens

In [7]:
keywords = []
pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
for token in doc:
    if(token.text in STOP_WORDS or token.text in punctuation):
        continue
    if(token.pos_ in pos_tag):
        keywords.append(token.text)

freq_word = Counter(keywords)
freq_word.most_common(5)

[('learning', 8), ('Machine', 4), ('study', 3), ('algorithms', 3), ('task', 3)]

### Normalization

In [8]:
max_freq = Counter(keywords).most_common(1)[0][1]
for word in freq_word.keys():
    freq_word[word] = (freq_word[word]/max_freq)

freq_word.most_common(5)

[('learning', 1.0),
 ('Machine', 0.5),
 ('study', 0.375),
 ('algorithms', 0.375),
 ('task', 0.375)]

### Weighing sentences

In [9]:
sent_strength = {}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_word.keys():
            if sent in sent_strength.keys():
                sent_strength[sent] += freq_word[word.text]
            else:
                sent_strength[sent] = freq_word[word.text]

print(sent_strength)

{Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task.: 4.125, Machine learning algorithms build a mathematical model of sample data, known as “training data”, in order to make predictions or decisions without being explicitly programmed to perform the task.: 4.625, Machine learning algorithms are used in the applications of email filtering, detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task.: 4.25, Machine learning is closely related to computational statistics, which focuses on making predictions using computers.: 2.625, The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.: 3.125, Data mining is a field of study within machine learning and focuses on exploratory data analysis through unsupervised learnin

### Summarizing the string

In [15]:
summary = nlargest(5, sent_strength, key=sent_strength.get)

# Convert summary to string
final_summary = [word.text for word in summary]
summary = ' '.join(final_summary)
print(summary)

Machine learning algorithms build a mathematical model of sample data, known as “training data”, in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Data mining is a field of study within machine learning and focuses on exploratory data analysis through unsupervised learning. Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.
